### Your name:

<pre> Siqi Liu </pre>

### Collaborators:

<pre> </pre>


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Open the housing data


In [2]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()





,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Build full pipeline for the data analysis following the example of the notebook.
 Hint: the main part requested to change is the algorithm used (Lasso regression)

If you want to learn more about the Lasso regression, see resources below:
- http://scikit-learn.org/stable/modules/linear_model.html#lasso
- https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/

#### Considerations for building pipeline:

- Split data into training and testing sets below.
- Convert all categorical data to one-hot vectors below
- Normalize all non-categorical data 
-  Perform Lasso-based regression using a variety of values for $\alpha$ between 0 and 1 via a grid search where  *housing_labels* is the output and all other features are the input (similar to as seen in lecture two.)

# Train/Test Split

In [3]:
np.random.seed(42)

In [4]:
# Divide by 1.5 to limit the number of income categories
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# Label those above 5 as 5
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [6]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set)
}).sort_index()
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [7]:
compare_props

,Overall,Stratified,Strat. %error
1.0,0.039826,0.039729,-0.243309
2.0,0.318847,0.318798,-0.015195
3.0,0.350581,0.350533,-0.013820
4.0,0.176308,0.176357,0.027480
5.0,0.114438,0.114583,0.127011


In [8]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Data Cleaning Pipelines

In [9]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

## *Impute Missing Values*

In [10]:
# Missing value per each column
housing.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        158
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

In [11]:
housing['total_bedrooms'].median()

433.0

In [12]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [13]:
housing_num = housing.drop('ocean_proximity', axis=1)

In [14]:
imputer.fit(housing_num)

SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbose=0)

In [15]:
imputer.statistics_ # Median values for all numerical columns

array([-118.51  ,   34.26  ,   29.    , 2119.5   ,  433.    , 1164.    ,
        408.    ,    3.5409])

In [16]:
X = imputer.transform(housing_num)

In [17]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042
1,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214
2,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621
3,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839
4,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347


In [18]:
housing_tr.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64

## *One-Hot Encoding*

In [19]:
housing['ocean_proximity'].value_counts(dropna=False)

<1H OCEAN     7276
INLAND        5263
NEAR OCEAN    2124
NEAR BAY      1847
ISLAND           2
Name: ocean_proximity, dtype: int64

In [20]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat = housing[['ocean_proximity']]
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot.toarray()

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

## *Add Attributes*

In [21]:
from sklearn.preprocessing import FunctionTransformer

# get the right column indices: safer than hard-coding indices 3, 4, 5, 6
rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

def add_extra_features(X, add_bedrooms_per_room=True):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if add_bedrooms_per_room:
        bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
        return np.c_[X, rooms_per_household, population_per_household,
                     bedrooms_per_room]
    else:
        return np.c_[X, rooms_per_household, population_per_household]

## *Pipelines*

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
        ('std_scaler', StandardScaler()),
        # Add normalization
        ('min_max_scaler', MinMaxScaler())
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [23]:
housing_num_tr.shape

(16512, 11)

In [24]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [25]:
housing_prepared.shape

(16512, 16)

# Training

## *Lasso with Alpha = 0*

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
from sklearn.linear_model import Lasso

lasso_reg_zero = Lasso(alpha = 0)

lasso_reg_zero.fit(housing_prepared, housing_labels)

Lasso(alpha=0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [28]:
from sklearn.metrics import mean_squared_error

In [29]:
lasso_zero_train_housing_labels = lasso_reg_zero.predict(housing_prepared)
lasso_zero_train_mse = mean_squared_error(housing_labels, lasso_zero_train_housing_labels)
lasso_zero_train_rmse = np.sqrt(lasso_zero_train_mse)
lasso_zero_train_rmse

68628.19819848923

In [30]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

In [31]:
lasso_zero_test_housing_labels = lasso_reg_zero.predict(X_test_prepared)
lasso_zero_test_mse = mean_squared_error(y_test, lasso_zero_test_housing_labels)
lasso_zero_test_rmse = np.sqrt(lasso_zero_test_mse)
lasso_zero_test_rmse

66911.98070857546

## *GridSearch for Alpha*

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

param_grid = [
    {'alpha': [n/100 for n in range(0,101,1)]}
  ]

lasso_reg = Lasso()

grid_search = GridSearchCV(lasso_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, ....84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [33]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'alpha': 1.0}
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)


In [34]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

69146.94962732709 {'alpha': 0.0}
69146.67840935856 {'alpha': 0.01}
69146.40741617409 {'alpha': 0.02}
69146.13664777631 {'alpha': 0.03}
69145.86610416786 {'alpha': 0.04}
69145.5957853514 {'alpha': 0.05}
69145.32569132956 {'alpha': 0.06}
69145.05582210496 {'alpha': 0.07}
69144.78617768023 {'alpha': 0.08}
69144.51675805802 {'alpha': 0.09}
69144.24756324093 {'alpha': 0.1}
69143.97859323163 {'alpha': 0.11}
69143.7098480327 {'alpha': 0.12}
69143.44132764678 {'alpha': 0.13}
69143.17303207649 {'alpha': 0.14}
69142.90496132446 {'alpha': 0.15}
69142.63711539326 {'alpha': 0.16}
69142.36949428555 {'alpha': 0.17}
69142.10209800392 {'alpha': 0.18}
69141.83492655099 {'alpha': 0.19}
69141.56797992934 {'alpha': 0.2}
69141.30125814161 {'alpha': 0.21}
69141.03476119036 {'alpha': 0.22}
69140.76848907824 {'alpha': 0.23}
69140.5024418078 {'alpha': 0.24}
69140.23661938167 {'alpha': 0.25}
69139.97102180243 {'alpha': 0.26}
69139.70564907267 {'alpha': 0.27}
69139.44050119497 {'alpha': 0.28}
69139.17557817194 {'

In [35]:
lasso_reg_final = grid_search.best_estimator_
lasso_reg_final

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [36]:
lasso_final_train_housing_labels = lasso_reg_final.predict(housing_prepared)
lasso_final_train_mse = mean_squared_error(housing_labels, lasso_final_train_housing_labels)
lasso_final_train_rmse = np.sqrt(lasso_final_train_mse)
lasso_final_train_rmse

68628.60780782861

In [37]:
lasso_final_test_housing_labels = lasso_reg_final.predict(X_test_prepared)
lasso_final_test_mse = mean_squared_error(y_test, lasso_final_test_housing_labels)
lasso_final_test_rmse = np.sqrt(lasso_final_test_mse)
lasso_final_test_rmse

66916.36758710597

## *Linear Regression*

In [38]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [39]:
lin_train_housing_labels = lin_reg.predict(housing_prepared)
lin_train_mse = mean_squared_error(housing_labels, lin_train_housing_labels)
lin_train_rmse = np.sqrt(lin_train_mse)
lin_train_rmse

68628.19819848923

In [40]:
lin_test_housing_labels = lin_reg.predict(X_test_prepared)
lin_test_mse = mean_squared_error(y_test, lin_test_housing_labels)
lin_test_rmse = np.sqrt(lin_test_mse)
lin_test_rmse

66911.98070857547

## *Coefficients Comparison*

In [41]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]

cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])

coefs = pd.DataFrame(index = ['RMSE (training)', 'RMSE (test)', 'intercept']
                             + num_attribs
                             + extra_attribs
                             + cat_one_hot_attribs,
                    columns = ['Lasso Final','Lasso w Alpha = 0','Linear'])

In [42]:
coefs_lasso_zero = [lasso_zero_train_rmse, lasso_zero_test_rmse]
coefs_lasso_zero.extend([lasso_reg_zero.intercept_])
coefs_lasso_zero.extend(lasso_reg_zero.coef_)

coefs['Lasso w Alpha = 0'] = coefs_lasso_zero

In [43]:
coefs_lasso_final = [lasso_final_train_rmse, lasso_final_test_rmse]
coefs_lasso_final.extend([lasso_reg_final.intercept_])
coefs_lasso_final.extend(lasso_reg_final.coef_)

coefs['Lasso Final'] = coefs_lasso_final

In [44]:
coefs_lin = [lin_train_rmse, lin_test_rmse]
coefs_lin.extend([lin_reg.intercept_])
coefs_lin.extend(lin_reg.coef_)

coefs['Linear'] = coefs_lin

In [45]:
#Set the display format to be scientific for ease of analysis
pd.options.display.float_format = '{:,.1f}'.format

coefs

,Lasso Final,Lasso w Alpha = 0,Linear
RMSE (training),"68,628.6","68,628.2","68,628.2"
RMSE (test),"66,916.4","66,912.0","66,912.0"
intercept,"219,233.9","210,688.4","244,150.8"
longitude,"-278,197.8","-279,113.9","-279,113.9"
latitude,"-248,727.0","-249,627.5","-249,627.5"
housing_median_age,"55,716.9","55,698.7","55,698.7"
total_rooms,"-29,691.4","-35,540.2","-35,540.2"
total_bedrooms,"111,270.1","110,764.1","110,764.1"
population,"-1,452,593.1","-1,461,768.1","-1,461,768.1"
households,"639,316.2","647,999.4","647,999.4"


In [46]:
np.absolute(lasso_reg_zero.intercept_) + np.sum(np.absolute(lasso_reg_zero.coef_)) 

4691458.715942057

In [47]:
np.absolute(lin_reg.intercept_) + np.sum(np.absolute(lin_reg.coef_)) 

4734811.168579877

In [48]:
np.absolute(lasso_reg_final.intercept_) + np.sum(np.absolute(lasso_reg_final.coef_)) 

4617157.7270190185

# Questions


Why is it necessary to normalize all continuous variables before performing Lasso? (OPTIONAL)

Since Lasso regression adds L1 regularization based on the sum of the absolute values of the coefficients, it's necessary for all variables to be normalized to the same range. Otherwise, variables with large ranges (e.g. total_rooms) would have much smaller coefficients than variables with smaller ranges (e.g. median_income), causing asymmteric weights on the variables during gradient descent.

### Conclusions
For what values of $\alpha$ does Lasso perform best? Does it perform as well on the housing data as the linear regressor from the lectures? Why do you think this is?

Lasso performs best with $\alpha$ = 1.0.

Comparing the RMSE between the final Lasso model and the Linear Regession model, we can see that the final Lasso model actually performs *worse* than the Linear Regression model on both the training set and the test set.

To understand why this is the case, I ran a Lasso model with $\alpha$ = 0, so there's no regularization, hence equivalent to a regular linear regression model. Interestingly enough, the fitted coefficients turned out to be different between the two models for the intercept and the one-hot encoded columns.

If we compare the sum of the absolute values of the coefficients bewteen the two Lasso models, we can see that the model with $\alpha$ = 0.0 has *higher* sum than the model with $\alpha$ = 1.0. This is expected, confirming that the L1 regularization worked. At the expense of doing so, the model with $\alpha$ = 1.0 actually underfit the data, so the performance is worse.

As $\alpha$ becomes larger and larger (approaches infinity), more and more coefficients would actually become zero. This is why Lasso regressions are often used to reduce the number of features when there are a lot of them present.

### Read appending B

- Reflect on your last data project, read appendix B. Then, write down a few of the checklist items that your last data project could have used. If you have not yet done a data project, then write down a few of the items that you found most interesting.


A few things my last data project could have used:

- Getting the Data
    - Sample a test set, put it aside and never look at it
- Explore the Data
    - Create a copy of the data for exploration
    - Study the type of distribution of each attribute
    - Study how you would solve the problem manually
    - Identify the promising transformations you maywant to apply
    - Identify extra data that would be useful
- Prepare the Data
    - Work on copies of the data
    - Write functions for data transformations
    - Aggregate features
    - Discretize continuous features
    - Feature scaling
- Short-List Promising Models
    - Train many quick and dirty models
    - Analyze the types of errors the models make
- Fine-Tune the System
    - Use random search over grid search
    - Try Ensemble methods